In [3]:
import numpy as np
from transformer import Attention, Attention_cache, Attention_init

ModuleNotFoundError: No module named 'transformer'

In [ ]:
B, T, C = 2, 4, 6

In [ ]:
rng = np.random.default_rng(2000)

``` python
@dataclass
class Attention_init:
    gamma: npt.NDArray[np.float64] # (1, 1, C) or (C, )
    beta: npt.NDArray[np.float64]  # (1, 1, C) or (C, )

    attentionHead: int             # H

    Wk: npt.NDArray[np.float64]    # (C, C) when C is hidden param or d_model
    Wq: npt.NDArray[np.float64]    # (C, C)
    Wv: npt.NDArray[np.float64]    # (C, C)

    merge_heads_bias: npt.NDArray[np.float64] # (1, 1, C) recommend

```

In [ ]:
attentionInit = Attention_init(
    gamma=  rng.random((1, 1, C), dtype = np.float64),
    beta= rng.random((1, 1, C), dtype=np.float64),
    attentionHead= 2,
    Wk=rng.random((1, C, C), np.float64),
    Wq = rng.random((1, C, C), np.float64),
    Wv = rng.random((1, C, C), np.float64),
    merge_heads_bias=rng.random((1, 1, C), np.float64),

)

In [ ]:
attention = Attention(attentionInit, "attentiondemo")

In [ ]:
def grad_check_transformer(forward, backward, eps=1e-6):
    rng = np.random.default_rng(1000)

    B, T, C = 2, 4, 6

    X = -rng.random((B, T, C), dtype=np.float64)
    G = -rng.random((B, T, C), dtype=np.float64) # upstream gradient

    # forward
    Y = forward(X)

    # analytical gradient
    dX = backward(G)

    # numerical gradient
    dX_num = np.zeros_like(X)
    it = np.nditer(X, flags=['multi_index'], op_flags=['readwrite'])

    while not it.finished:
        idx = it.multi_index
        old = X[idx]

        X[idx] = old + eps
        Yp = forward(X)
        Lp = np.sum(Yp * G)

        X[idx] = old - eps
        Ym = forward(X)
        Lm = np.sum(Ym * G)

        X[idx] = old
        dX_num[idx] = (Lp - Lm) / (2 * eps)

        it.iternext()

    # compare

    max_abs = np.max(np.abs(dX - dX_num))
    rel = max_abs / (np.max(np.abs(dX) + np.abs(dX_num)) + 1e-22)


    print("max_abs_diff:", max_abs)
    print("relative_diff:", rel)

In [ ]:
grad_check_transformer(forward=attention.forward_train, backward=attention.backward, eps=1e-6)

TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

In [ ]:
X = rng.random((B, T, C), dtype=np.float64)

In [ ]:
attention.forward_train(X)

In [ ]:
attention.Q_split.shape

(2, 2, 4, 3)

In [ ]:
attention.K_split.shape

(2, 2, 4, 3)

In [ ]:
attention.scores

array([[[[3.83496518, 3.91845587, 5.37339579, 2.68240767],
         [3.15911386, 3.19127673, 4.57904803, 2.38957341],
         [3.9229944 , 3.98181699, 5.67189336, 2.95385506],
         [1.66152568, 1.62701761, 2.47209586, 1.32392322]],

        [[5.3130294 , 4.5760051 , 5.00512947, 1.41665579],
         [4.86950489, 4.25931395, 4.71681246, 1.34907468],
         [6.81921798, 5.99585957, 6.66014132, 1.97908475],
         [3.57308366, 3.18681124, 3.57748393, 1.08859133]]],


       [[[3.93651729, 2.97079215, 4.1935504 , 2.77173733],
         [1.75224639, 1.43867577, 1.91423718, 1.27401304],
         [3.4592222 , 2.69459272, 3.70621879, 2.39239388],
         [2.47423822, 1.9542011 , 2.6357916 , 1.57781742]],

        [[5.02852816, 1.76379282, 4.32497193, 2.32321157],
         [3.34775732, 1.30827139, 3.05638825, 1.70954982],
         [5.07391599, 1.81862409, 4.44611525, 2.47981239],
         [3.50578726, 1.29242308, 3.08662797, 1.66277238]]]])

In [ ]:
attention.output.shape

(2, 2, 4, 3)

In [ ]:
attention.merge_heads(attention.output).shape

(2, 4, 6)

In [ ]:
G = -rng.random((B, T, C), dtype=np.float64)

NameError: name 'rng' is not defined